In [13]:
import pandas as pd
import requests
import time

In [14]:
API_KEY = 'YOUR_API_KEY'

In [15]:
route_info = """
1. ISBT Maharana Pratap Bus (T)
2. ISBT Ring Road
3. Metro Train Depot
4. Shyam Giri Mandir
5. Shastri Park
6. Shastri Park
7. Dharampura
8. Seelampur Metro Station
9. Seelam Pur Petrol Pump
10. Welcome Metro Station
11. Kanti Nagar Extension
12. Jharkhandi
13. East Azad Nagar
14. Swarn Cinema
15. Krishna Nagar A Block
16. Hans Apartment
17. Arjun Nagar
18. Radheypuri
19. Jagat Puri A-Block
20. Jagatpuri F-Block
21. Hasan Pur Depot (Patparganj Depot)
22. Hassanpur Village
23. Gazipur Depot
24. Anand Vihar Terminal
"""

In [19]:
stops_list = [line.split('. ', 1)[1].strip() for line in route_info.strip().split('\n') if line.strip()]


In [22]:


def get_coordinates(stop,city = 'Delhi'):
    url = f'https://api.tomtom.com/search/2/geocode/{stop}, {city}.json'
    params = {
     'key':API_KEY
    }
    max_retry = 3
    for attempt in range(max_retry):
        try:
            response = requests.get(url,params = params,timeout = 10)
            response.raise_for_status()
            data = response.json()
            if data['results']:
                latitude = data['results'][0]['position']['lat']
                longitude = data['results'][0]['position']['lon']
                return latitude,longitude
            else:
                print(f"No reults found for {stop}")
                return None,None
        except requests.exception.RequestException as e:
             if attempt < max_retry - 1:
                 print(f"Error occoured: {e}. Retrying in 5 seconds...")
                 time.sleep(5)
             else   :
                 print(f"failed to get coordinates for {stop} after {max_retry} attempt")
                 return None,None             


In [24]:
coordinates = {}

for stop in stops_list:
    lat,lon = get_coordinates(stop)
    coordinates[stop] = {'latitude':lat,'longitude':lon}
    time.sleep(1)

No reults found forSeelam Pur Petrol Pump


In [37]:
def get_traffic(latitude,longitude):
    url = f'https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json'
    
    params = {
        'point': f'{latitude},{longitude}',
        'key': API_KEY
    }
    response = requests.get(url,params = params)
    if response.status_code  == 200:
        data = response.json()
        flow_data = {
            'Current Speed(km/h)' : data['flowSegmentData']['currentSpeed'],
            'Free Flow Speed(km/h)' : data['flowSegmentData']['freeFlowSpeed'],
            'Confidence' : data['flowSegmentData']['confidence'],
            'Current Travel Time(s)' : data['flowSegmentData']['currentTravelTime'],
            'Free Flow Time(s)' : data['flowSegmentData']['freeFlowTravelTime'],
            'Coordinates' : data['flowSegmentData']['coordinates']['coordinate']
        }
        return flow_data
    else:
        print(f"Error: Unable to fetch traffic data. Status code: {response.status_code}")
        return None   
          
    

In [38]:
def get_routes(start_lat,start_lon,end_lat,end_lon):
    url = f'https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{end_lat},{end_lon}/json'
    params = {
    'traffic': 'true',
    'key': API_KEY
     }
    response = requests.get(url,params=params)
    if response.status_code  == 200:
        data = response.json()
        summary = data['routes'][0]['summary']
        route_data = {
            'Distance (km)' :summary['lengthInMeters']/1000,
            'Travel Time (Min)' : summary['travelTimeInSeconds']/60,
            'Traffic Delay (Min)': summary['trafficDelayInSeconds']/60
            }
        return route_data
    else:
        print(f"Error: Unable to fetch route data. Status code: {response.status_code}")
        return None 


In [39]:
all_flow_data = []
all_route_data = []

for location,coord in coordinates.items():
    flow_data = get_traffic(coord['latitude'],coord['longitude'])
    if flow_data:
        flow_data['Location'] = location
        all_flow_data.append(flow_data)
    time.sleep(1)    

Error: Unable to fetch traffic data. Status code: 400


In [45]:
location = list(coordinates.keys())
for i in range(len(location) - 1):
    start = coordinates[location[i]]
    end = coordinates[location[i+1]]
    route_data = get_routes(start['latitude'],start['longitude'],end['latitude'],end['longitude'])
    if route_data:
        route_data['From'] = location[i]
        route_data['To'] = location[i+1]
        all_route_data.append(route_data)
    time.sleep(1)    

Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400
Error: Unable to fetch route data. Status code: 400


In [47]:
df_flow = pd.DataFrame(all_flow_data)
df_route = pd.DataFrame(all_route_data)

In [49]:
df_flow.to_csv('Flow.csv',index = False)
df_route.to_csv('Route.csv',index = False)

In [56]:
df_combined = pd.merge(df_flow,df_route,left_on = 'Location',right_on = 'From',how = 'outer')
df_combined = df_combined.dropna()
#data = data.dropna()
df_combined.to_csv('FinalData.csv',index = False)


In [55]:
df_combine

,Current Speed(km/h),Free Flow Speed(km/h),Confidence,Current Travel Time(s),Free Flow Time(s),Coordinates,Location,Distance (km),Travel Time (Min),Traffic Delay (Min),From,To
0,33,33,1,250,250,"[{'latitude': 28.642455847925508, 'longitude':...",ISBT Maharana Pratap Bus (T),15.292,30.016667,1.716667,ISBT Maharana Pratap Bus (T),ISBT Ring Road
1,25,34,1,207,152,"[{'latitude': 28.584116432419023, 'longitude':...",ISBT Ring Road,11.663,17.166667,0.000000,ISBT Ring Road,Metro Train Depot
2,39,39,1,223,223,"[{'latitude': 28.67172543890212, 'longitude': ...",Metro Train Depot,34.652,65.983333,0.000000,Metro Train Depot,Shyam Giri Mandir
3,39,39,1,1974,1974,"[{'latitude': 28.55796490301944, 'longitude': ...",Shyam Giri Mandir,49.196,68.383333,0.000000,Shyam Giri Mandir,Shastri Park
4,21,21,1,780,780,"[{'latitude': 28.65953208073625, 'longitude': ...",Shastri Park,44.193,63.200000,0.000000,Shastri Park,Dharampura
5,36,36,1,1710,1710,"[{'latitude': 28.608065896274542, 'longitude':...",Dharampura,7.882,17.400000,0.000000,Dharampura,Seelampur Metro Station
7,36,36,1,345,345,"[{'latitude': 28.619433072411205, 'longitude':...",Welcome Metro Station,9.999,21.116667,0.000000,Welcome Metro Station,Kanti Nagar Extension
8,36,36,1,1228,1228,"[{'latitude': 28.58586257082214, 'longitude': ...",Kanti Nagar Extension,29.622,54.150000,0.000000,Kanti Nagar Extension,Jharkhandi
9,29,29,1,560,560,"[{'latitude': 28.652991520640715, 'longitude':...",Jharkhandi,1.019,5.600000,0.000000,Jharkhandi,East Azad Nagar
10,20,20,1,603,603,"[{'latitude': 28.664175039379867, 'longitude':...",East Azad Nagar,26.721,44.783333,0.000000,East Azad Nagar,Swarn Cinema
